In [1]:
import numpy as np
import factorgraph as fg

In [15]:
reload(fg)

<module 'factorgraph' from 'factorgraph.py'>

In [2]:
def psi(n,i,j):
    pot=np.zeros((n,n))
    for xi in range(n):
        for yj in range(n):
            if not((xi==j)!=(yj==i)):
                pot[xi,yj]=1
    return pot                     

In [3]:
# Make an empty graph
g = fg.Graph()
n=5
np.random.seed(5)
P=np.random.randint(100,size=(n,n));
phi=np.sqrt(P)

# Add some discrete random variables (RVs)
for i in range(n):
    g.rv('x'+str(i),n)
for i in range(n):
    g.rv('y'+str(i),n)
# Add some factors, unary and binary
# g.factor(['a'], potential=np.array([0.3, 0.7]))
# g.factor(['b', 'a'], potential=np.array([
#         [0.3, 0.8],
#         [0.4, 0.6],
#         [0.2, 0.8],
# ]))

for i in range(n):
    g.factor(['x'+str(i)],potential=phi[i,:])

for i in range(n):
    g.factor(['y'+str(i)],potential=phi[:,i])

for i in range(n):
    for j in range(n):
        g.factor(['x'+str(i),'y'+str(j)], potential=psi(n,i,j))


In [4]:
# Run (loopy) belief propagation (LBP)
iters, converged = g.lbp(normalize=True,max_iters=500)
print 'LBP ran for %d iterations. Converged = %r' % (iters, converged)
print

LBP ran for 500 iterations. Converged = False



In [5]:
# Print out the final messages from LBP
g.print_messages()
# print

# Print out the final marginals
g.print_rv_marginals(normalize=True)

Current outgoing messages:
	f(x0) -> x0 	[0.25423913 0.2256691  0.19956745 0.10220798 0.21831633]
	f(x1) -> x1 	[0.09328559 0.2596961  0.17137658 0.18064678 0.29499495]
	f(x2) -> x2 	[0.08409645 0.27709932 0.1231046  0.23140173 0.2842979 ]
	f(x3) -> x3 	[0.13920672 0.17770705 0.23508434 0.23201121 0.21599068]
	f(x4) -> x4 	[0.1957959  0.15642829 0.26175459 0.26485239 0.12116883]
	f(y0) -> y0 	[0.3621315  0.10294226 0.09629367 0.18911701 0.24951556]
	f(y1) -> y1 	[0.23530002 0.20978311 0.23226377 0.17672623 0.14592688]
	f(y2) -> y2 	[0.22430676 0.1492311  0.11123029 0.25201293 0.26321892]
	f(y3) -> y3 	[0.11530299 0.15788512 0.20985461 0.2496383  0.26731898]
	f(y4) -> y4 	[0.22056188 0.23089472 0.23089472 0.20812569 0.10952298]
	f(x0, y0) -> x0 	[0.48281333 0.12929667 0.12929667 0.12929667 0.12929667]
	f(x0, y0) -> y0 	[0.29611581 0.17597105 0.17597105 0.17597105 0.17597105]
	f(x0, y1) -> x0 	[0.20021236 0.19915055 0.20021236 0.20021236 0.20021236]
	f(x0, y1) -> y1 	[0.15070249 0.212324

In [6]:
0.25423913*0.27919782*0.29611581

0.02101919175547727

In [6]:
belief_x=np.array([x[1] for x in sorted(g.rv_marginals(normalize=False))[0:n]]);
belief_y=np.array([x[1] for x in sorted(g.rv_marginals(normalize=False))[n:2*n]]);

In [7]:
belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
for i in range(n):
    for j in range(n):
        belief_xy[i,j,:,:]=np.outer(belief_x[i,:],belief_y[j,:])
for i in range(n):
    for j in range(n):
        belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])

In [12]:
np.sum(belief_xy[0,0],axis=1)

array([0.59123578, 0.13979419, 0.13048145, 0.02856918, 0.1099194 ],
      dtype=float128)

In [35]:
def bfe(P,belief_xy,belief_x,belief_y):
        
    t1=t2=t3=t4=0
    n=P.shape[0]
#     belief_x=np.zeros((n,n),dtype=float)
#     belief_y=np.zeros((n,n),dtype=float)
    phi=np.sqrt(P)
    
#     belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=np.outer(belief_x[i,:],belief_y[j,:])/(m_x2y[i,j]*m_y2x[j,i])
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])

    
#     for i in range(n):
#         belief_x[i,:]=np.sum(belief_xy[i,0],axis=1)
#         belief_y[i,:]=np.sum(belief_xy[0,i],axis=0)
        
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if belief_xy[i,j,xi,yj]!=0 and not((xi==j)!=(yj==i)):
                        t1+=belief_xy[i,j,xi,yj]*np.log(phi[i,xi]*phi[yj,j])
                        t2+=belief_xy[i,j,xi,yj]*np.log(belief_xy[i,j,xi,yj])
    
    for i in range(n):
        for xi in range(n):
            if belief_x[i,xi]!=0:
                t3+=belief_x[i,xi]*np.log(belief_x[i,xi])
    
    for j in range(n):
        for yj in range(n):
            if belief_y[j,yj]!=0:
                t4+=belief_y[j,yj]*np.log(belief_y[j,yj])
    print('t1', -t1)    
    print('t2', t2)
    print('t3', -(n-1)*t3)
    print('t4', -(n-1)*t4)
    energy=-t1+t2-(n-1)*t3-(n-1)*t4
    print(energy)
    return np.exp(-energy)

In [36]:
perm_bethe=bfe(P,belief_xy,belief_x,belief_y)

('t1', -71.31864155555759242)
('t2', -46.94460620257107001)
('t3', 27.369792474917094)
('t4', 27.369792474917094)
-63.52366280829447355


In [37]:
perm_bethe

3.872364480928070321e+27

In [38]:
def permanent(A):
    assert(len(A.shape)==2),"Input should be a 2D matrix"
    assert(A.shape[0]==A.shape[1]),"Input should be a square matrix"
    
    perm=np.float128(0)
    n=A.shape[1]
    C=2**n
    
    for k in range(1,C):
        rowsumprod=np.float128(1)
        chi=np.array(list(np.binary_repr(k))).astype(np.float128)
        for m in range(0,n):
            rowsum=np.float128(0)
            
            for p in range(0,len(chi)):
                rowsum += chi[p]*A[m,len(chi)-p-1]
            
            rowsumprod *= rowsum
            
        perm += ((-1)**(n-chi.sum()))*rowsumprod
    
    return perm

In [40]:
perm=permanent(P);perm

47584483202.0

In [4]:
np.sum([0.25423913, 0.2256691, 0.19956745, 0.10220798, 0.21831633])

0.9999999900000001